### Benchmarking a Method with SpikeSorting.jl

You can use SpikeSorting.jl to calculate quantitive metrics of the accuracy of the method. To do this, we need 1) ground truth data sets and 2) measures of "accuracy." To find the ground truth datasets, we will simulate extracellular potentials created by tens of thousands of pyramidal neurons. To measure performance, we will calculate the false positives attributable to detection and clustering steps, false negatives attributable to the detection threshold and overlap of multiple potentials, and the true positive percentage. These metrics as described in detail here:

http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3123734/pdf/nihms303529.pdf

In [ ]:
using SpikeSorting, JNeuron, DSP

In [ ]:
# We will generate our ground truth dataset with JNeuron

#Import Neuron and assign HH and Passive channels everywhere
myimport=input("/home/nicolelislab/JNeuron/examples/data/cell2.asc");
blank_neuron=instantiate(myimport);
set_nsegs!(blank_neuron);
template_neuron=add(blank_neuron,(HH(),Passive()));

#Find current at each node when this neuron fires an action potential
mynetwork=Network(template_neuron,15.0)
i_nodes=JNeuron.runc(mynetwork,true);
i_nodes=JNeuron.runc(mynetwork,false);

#Create extracellular electrode
mye=Extracellular([500.0,125.0,0.0])

#Find the voltage waveforms of every neuron in a 30000 neuron population
neuron_num=30000
voltage_waveforms=JNeuron.nete(template_neuron,i_nodes,mye,neuron_num);

#Find potential at extracellular electrode, and time stamps of nearby neurons
sim_time=30000.0 #30 seconds
(voltage_extracellular, time_stamps)=JNeuron.extrap(voltage_waveforms,sim_time);

#Remove LFP with high pass filter
r=Highpass(300, fs=40000)
dm = Butterworth(4)
fv1=filt(digitalfilter(r,dm),voltage_extracellular);

In [ ]:
#Benchmark with SpikeSort.jl
detect=DetectPower();
cluster=ClusterOSort();
align=AlignMin();
feature=FeatureTime();
reduce=ReductionNone();
thres=ThresholdMean();
s1=create_multi(detect,cluster,align,feature,reduce,thres,1);

fv=fv1[1000:2:end]
ss=SpikeSorting.benchmark(fv,time_stamps,s1[1],15.0,20000)